# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.297948417728                   -0.85    5.9         
  2   -8.300178079404       -2.65       -1.25    1.9    129ms
  3   -8.300432538358       -3.59       -1.85    1.8    266ms
  4   -8.300461013427       -4.55       -2.68    1.9    122ms
  5   -8.300464347725       -5.48       -3.16    2.6    148ms
  6   -8.300464498272       -6.82       -3.30   11.8    287ms
  7   -8.300464555132       -7.25       -3.45    2.0    131ms
  8   -8.300464596846       -7.38       -3.60    1.1    124ms
  9   -8.300464632048       -7.45       -3.83    2.0    167ms
 10   -8.300464638003       -8.23       -3.95    8.0    234ms
 11   -8.300464643480       -8.26       -4.26    3.5    181ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.63233384256                   -0.70    5.9         
  2   -16.67846275651       -1.34       -1.13    1.9    296ms
  3   -16.67921748458       -3.12       -1.86    3.1    318ms
  4   -16.67928023389       -4.20       -2.63    3.2    356ms
  5   -16.67928591898       -5.25       -3.06    3.9    356ms
  6   -16.67928619646       -6.56       -3.49    3.2    311ms
  7   -16.67928621811       -7.66       -4.13    2.2    301ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32551919233                   -0.56    7.6         
  2   -33.33435300521       -2.05       -1.00    1.2    1.04s
  3   -33.33594293870       -2.80       -1.73    7.5    1.45s
  4   -33.33614686420       -3.69       -2.64    4.2    1.28s
  5   -33.33684221457       -3.16       -2.32    6.8    1.68s
  6   -33.33687197741       -4.53       -2.40    1.8    989ms
  7   -33.33694308225       -4.15       -3.47    1.8    1.09s
  8   -33.33694355017       -6.33       -3.61    4.5    1.56s
  9   -33.33694377954       -6.64       -4.45    1.6    1.01s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298490097508                   -0.85    4.9         
  2   -8.300269165852       -2.75       -1.59    1.2   87.9ms
  3   -8.300435673268       -3.78       -2.69    3.8    131ms
  4   -8.300440011388       -5.36       -2.55    3.1    144ms
  5   -8.300464335431       -4.61       -3.33    1.0   84.5ms
  6   -8.300464594662       -6.59       -3.78    2.6    117ms
  7   -8.300464640636       -7.34       -4.33    2.1    105ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32416126675                   -0.56    7.4         
  2   -33.32416408331       -5.55       -1.27    1.0    827ms
  3   -21.73515777082   +    1.06       -0.53    5.8    1.66s
  4   -33.27624962069        1.06       -1.62    6.9    1.77s
  5   -33.28386917000       -2.12       -1.37    2.6    1.18s
  6   -33.33329454826       -1.31       -2.09    2.6    1.19s
  7   -33.14815962213   +   -0.73       -1.43    3.9    1.53s
  8   -33.32893403184       -0.74       -2.07    5.4    1.46s
  9   -33.33614354188       -2.14       -2.57    2.0    960ms
 10   -33.33645939168       -3.50       -2.39    3.4    1.28s
 11   -33.33689380119       -3.36       -3.07    2.2    1.10s
 12   -33.33692364578       -4.53      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.